<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/%E3%80%8CGPT4Dev_ch14%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 14 GPTs Action 開發

## 14-2 使用TDX平台查詢台鐵資訊

### 查詢所有車站基本資料

In [ ]:
!pip install rich
from rich import print as pprint
import requests

In [ ]:
headers = {'user-agent':'Mozilla/5.0'}
api_base_url = (
    'https://tdx.transportdata.tw/api/basic/v3/'
    'Rail/TRA/'
)

In [ ]:
res = stations = requests.get(
    f"{api_base_url}Station",
    headers=headers,
)
print(res.status_code)

In [ ]:
json_data = res.json()

In [ ]:
pprint(json_data)

In [ ]:
res = requests.get(
    f"{api_base_url}Station"
    "?$select=stationName,stationID",
    headers=headers,
)
print(res.status_code)

In [ ]:
json_data = res.json()
stations = {}
for station in json_data['Stations']:
    station_name = station['StationName']['Zh_tw']
    station_id = station['StationID']
    stations[station_name] = station_id
pprint(stations)

### 指定起迄站代號查詢時刻表

In [ ]:
import time
now = time.localtime()
date = f"{now.tm_year}-{now.tm_mon:02d}-{now.tm_mday:02d}"
print(date)

In [ ]:
start_station = '2240'  # 龍井站代號
end_station = '2250'    # 大肚站代號
res = stations = requests.get(
    f"{api_base_url}DailyTrainTimetable/OD/"
    f"{start_station}"
    "/to/"
    f"{end_station}"
    "/"
    f"{date}",
    headers=headers,
)
print(res.status_code)

In [ ]:
json_data = res.json()
pprint(json_data)

In [ ]:
timetables = []
for timetable in json_data['TrainTimetables']:
    train_no = timetable['TrainInfo']['TrainNo']
    stop_times = timetable['StopTimes']
    start_station = stop_times[0]['StationName']['Zh_tw']
    departure_Time = stop_times[0]['DepartureTime']
    end_station = stop_times[1]['StationName']['Zh_tw']
    arrive_Time = stop_times[1]['ArrivalTime']
    timetables.append({
        'train_no': train_no,
        'start_station': start_station,
        'departure_Time': departure_Time,
        'end_station': end_station,
        'arrive_Time': arrive_Time
    })
pprint(timetables[0])

### 以驗證身分方式使用 API


In [ ]:
!git clone https://github.com/FlagTech/tdx.git

In [ ]:
from google.colab import userdata
from tdx import TDX
tdx_client = TDX(
    userdata.get('TDX_ID'),
    userdata.get('TDX_SECRET')
)

In [ ]:
json_data = tdx_client.get_json(
    f"{api_base_url}Station"
    "?$select=stationName,stationID",
)

In [ ]:
stations = {}
for station in json_data['Stations']:
    station_name = station['StationName']['Zh_tw']
    station_id = station['StationID']
    stations[station_name] = station_id
pprint(stations)

In [ ]:
start_station = '1000'
end_station = '2170'
json_data = tdx_client.get_json(
    f"{api_base_url}DailyTrainTimetable/OD/"
    f"{start_station}"
    "/to/"
    f"{end_station}"
    "/"
    f"{date}"
)

In [ ]:
timetables = []
for timetable in json_data['TrainTimetables']:
    train_no = timetable['TrainInfo']['TrainNo']
    stop_times = timetable['StopTimes']
    start_station = stop_times[0]['StationName']['Zh_tw']
    departure_Time = stop_times[0]['DepartureTime']
    end_station = stop_times[1]['StationName']['Zh_tw']
    arrive_Time = stop_times[1]['ArrivalTime']
    timetables.append({
        'train_no': train_no,
        'start_station': start_station,
        'departure_Time': departure_Time,
        'end_station': end_station,
        'arrive_Time': arrive_Time
    })

for train in timetables:
    pprint(train)

In [ ]:
timetables = sorted(
    timetables,
    key=lambda timetable:timetable['departure_Time']
)

for train in timetables:
    pprint(train)